# Setup

In [1]:
# import packages
%run ../global_packages.py

# get the global parameters
%run ../global_pars.py

# import your local functions
sys.path.insert(1, '../')
from global_functions import *

# make sure the figures plot inline rather than at the end
%matplotlib inline

# Paths

In [2]:
figpath = '../figures/'

# SLA Detrending

In [ ]:
inpath = '../data_processing/2_SLA/raw_data/concatenated/'
infn = 'sla_dt_global_twosat_phy_l4__vDT2018_19930101_to_20200603.nc'
ds = xr.open_dataset(inpath + infn)

# Subset ------------------------------------------------#
lat_slice = slice(lat_bounds[0], lat_bounds[1])
lon_slice = slice(lon_bounds[0], lon_bounds[1])
time_slice = slice(ts,te)

ds = ds.sel(lat=lat_slice,lon=lon_slice, time = time_slice)

# Get daily sla
daily_sla = ds['sla']
lat = daily_sla.lat.values
lon = daily_sla.lon.values

# stack lat and lon into a single dimension called allpoints
stacked = daily_sla.stack(allpoints=['lat','lon'])

# set places where there are nans to zero since polyfit can't deal with them
stacked_nonan = stacked.fillna(0)

# convert date to a number to polyfit can handle it
datenum = dates.date2num(stacked_nonan.time)
daily_sla_slope, daily_sla_intercept = np.polyfit(datenum, stacked_nonan, 1)

#reshape the data
daily_sla_slope = np.reshape(daily_sla_slope, daily_sla.shape[1:3])
daily_sla_intercept = np.reshape(daily_sla_intercept, daily_sla.shape[1:3])

# define a function to compute a linear trend of a timeseries
def linear_detrend(y):
    x = dates.date2num(y.time)
    m, b = np.polyfit(x, y, 1)
    # we need to return a dataarray or else xarray's groupby won't be happy
    return xr.DataArray(y - (m*x + b))

# apply the function over allpoints to calculate the trend at each point
daily_sla_dtrnd = stacked_nonan.groupby('allpoints').apply(linear_detrend)
# unstack back to lat lon coordinates
daily_sla_dtrnd = daily_sla_dtrnd.unstack('allpoints')

# fill all points we set originally to zero back to nan
daily_sla_dtrnd = daily_sla_dtrnd.where(~np.isnan(daily_sla))

# find global means for comparison
stacked = daily_sla.stack(allpoints=['lat','lon'])
global_mean = stacked.mean(dim='allpoints',skipna=True)
# global_mean.plot()

# find global means for comparison 
stacked = daily_sla_dtrnd.stack(allpoints=['lat','lon'])
global_mean_dtrnd = stacked.mean(dim='allpoints',skipna=True)
# global_mean_dtrnd.plot()
# plt.legend


fig = plt.figure(figsize=(6,3),dpi=200)
plt.plot(global_mean.time,global_mean)
plt.plot(global_mean_dtrnd.time,global_mean_dtrnd, label = 'Detrended')
plt.legend()
plt.title('Indian Ocean Sea Level Anomaly')
plt.ylabel('Sea Level Anomaly ($m$)')
plt.xlabel('Time')

# plt.savefig(figpath + 'Fig_sla_detrending.pdf', format='pdf', dpi = 400)

# PDFs

In [ ]:
#WOD
ds_WOD = xr.open_dataset('../data_processing/1_WOD_Coastal/wod_coastal_processed.nc')

# DMI
ds_DMI= xr.open_dataset('../data_processing/3_DMI/dmi_processed.nc')

posIODyears = list(np.array(ds_DMI.pos_IOD_years))
sposIODyears = list(np.array(ds_DMI.spos_IOD_years))
negIODyears = list(np.array(ds_DMI.neg_IOD_years))
neuIODyears = list(np.array(ds_DMI.neu_IOD_years)) 

depth = '50_200'
Depth = '50-200'
############################################################################################################
# Western Arabian Sea
############################################################################################################
doxy = ds_WOD['doxy_wAS_' + depth]
lat = ds_WOD.lat_wAS
lon = ds_WOD.lon_wAS
t = ds_WOD.time_wAS

# positive IOD years -------------------------------------------# 
posIODdata_wAS,_,_,posIODtime_wAS,posIODmon_wAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             posIODyears, region = 'wAS')
# strong positive IOD years -------------------------------------------# 
sposIODdata_wAS,_,_,sposIODtime_wAS,sposIODmon_wAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             sposIODyears, region = 'wAS')

# negative IOD years -------------------------------------------# 
negIODdata_wAS,_,_,_,negIODmon_wAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             negIODyears, region = 'wAS')

# neutral IOD years -------------------------------------------# 
neuIODdata_wAS,_,_,_,neuIODmon_wAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             neuIODyears, region = 'wAS')

############################################################################################################
# Eastern Arabian Sea
############################################################################################################
doxy = ds_WOD['doxy_eAS_' + depth]
lat = ds_WOD.lat_eAS
lon = ds_WOD.lon_eAS
t = ds_WOD.time_eAS

# positive IOD years -------------------------------------------# 
posIODdata_eAS,_,_,_,posIODmon_eAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             posIODyears, region = 'eAS')

# strong positive IOD years -------------------------------------------# 
sposIODdata_eAS,_,_,_,sposIODmon_eAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             sposIODyears, region = 'eAS')

# negative IOD years -------------------------------------------# 
negIODdata_eAS,_,_,_,negIODmon_eAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             negIODyears, region = 'eAS')

# neutral IOD years -------------------------------------------# 
neuIODdata_eAS,_,_,_,neuIODmon_eAS,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             neuIODyears, region = 'eAS')

############################################################################################################
# Western Bay of Bengal
############################################################################################################
doxy = ds_WOD['doxy_wBoB_' + depth]
lat = ds_WOD.lat_wBoB
lon = ds_WOD.lon_wBoB
t = ds_WOD.time_wBoB

# positive IOD years -------------------------------------------# 
posIODdata_wBoB,_,_,_,posIODmon_wBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             posIODyears, region = 'wBoB')
# strong positive IOD years -------------------------------------------# 
sposIODdata_wBoB,_,_,_,sposIODmon_wBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             sposIODyears, region = 'wBoB')
# negative IOD years -------------------------------------------# 
negIODdata_wBoB,_,_,_,negIODmon_wBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             negIODyears, region = 'wBoB')

# neutral IOD years -------------------------------------------# 
neuIODdata_wBoB,_,_,_,neuIODmon_wBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             neuIODyears, region = 'wBoB')

############################################################################################################
# Eastern Bay of Bengal
############################################################################################################
doxy = ds_WOD['doxy_eBoB_' + depth]
lat = ds_WOD.lat_eBoB
lon = ds_WOD.lon_eBoB
t = ds_WOD.time_eBoB

# positive IOD years -------------------------------------------# 
posIODdata_eBoB,_,_,_,posIODmon_eBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             posIODyears, region = 'eBoB')
# strong positive IOD years -------------------------------------------# 
sposIODdata_eBoB,_,_,_,sposIODmon_eBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             sposIODyears, region = 'eBoB')
# negative IOD years -------------------------------------------# 
negIODdata_eBoB,_,_,_,negIODmon_eBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             negIODyears, region = 'eBoB')

# neutral IOD years -------------------------------------------# 
neuIODdata_eBoB,_,_,_,neuIODmon_eBoB,_ = IOD_year_group_WOD(doxy,
                                             lat,lon,t,
                                             IODyear_begin,IODyear_end,
                                             neuIODyears, region = 'eBoB')

# Make Histograms 
# hatch_mask = min_mean > hyp_thresh
x = [1,2,3,4]

fig = plt.figure(figsize=(8.7 / 2.54, 1.5), dpi=400)

params = {'legend.fontsize': 3,
         'axes.labelsize': 3.5,
         'axes.titlesize': 5.5,
         'xtick.labelsize':3,
         'ytick.labelsize':3,
         'axes.linewidth':0.35,
         'xtick.major.width':0.25,
         'xtick.major.size':1.5,
         'ytick.major.width':0.25,
         'ytick.major.size':1.5}
pylab.rcParams.update(params)

text_fz = 4.5
sq_sz = 20
lfz = 3.75
width = 0.2
lw = 0.75

# clrs = ['darkgreen','darkorange','navy','darkorchid', 'darkgreen']
clrs = ['darkgreen','navy','darkorange','darkorchid', 'darkgreen']
labels = ['Neutral IOD Phases','Positive IOD Phases','Negative IOD Phases','Hypoxia']
letters = ['a','b','c','d','e','f','g','h']

regions = ['WAS','EAS','WBoB','EBoB']

####################################################################################################

data_wAS = [neuIODdata_wAS,posIODdata_wAS,negIODdata_wAS,sposIODdata_wAS]
data_eAS = [neuIODdata_eAS,posIODdata_eAS,negIODdata_eAS,sposIODdata_eAS]
data_wBoB = [neuIODdata_wBoB,posIODdata_wBoB,negIODdata_wBoB,sposIODdata_wBoB]
data_eBoB = [neuIODdata_eBoB,posIODdata_eBoB,negIODdata_eBoB,sposIODdata_eBoB]

time_data_wAS = [neuIODmon_wAS,posIODmon_wAS,negIODmon_wAS,sposIODmon_wAS]
time_data_eAS = [neuIODmon_eAS,posIODmon_eAS,negIODmon_eAS,sposIODmon_eAS]
time_data_wBoB = [neuIODmon_wBoB,posIODmon_wBoB,negIODmon_wBoB,sposIODmon_wBoB]
time_data_eBoB = [neuIODmon_eBoB,posIODmon_eBoB,negIODmon_eBoB,sposIODmon_eBoB]

cnt = 1
for pp in range(4):

    if pp==0:
        sdata = data_wAS[0]
        pdata = data_wAS[1]
        ndata = data_wAS[2]
        spdata = data_wAS[3]
        
        stdata = time_data_wAS[0]
        ptdata = time_data_wAS[1]
        ntdata = time_data_wAS[2]
        sptdata = time_data_wAS[3]

    elif pp==1:
        sdata = data_eAS[0]
        pdata = data_eAS[1]
        ndata = data_eAS[2]
        spdata = data_eAS[3]
        
        stdata = time_data_eAS[0]
        ptdata = time_data_eAS[1]
        ntdata = time_data_eAS[2]
        sptdata = time_data_eAS[3]

    elif pp==2:
        sdata = data_wBoB[0]
        pdata = data_wBoB[1]
        ndata = data_wBoB[2]
        spdata = data_wBoB[3]
        
        stdata = time_data_wBoB[0]
        ptdata = time_data_wBoB[1]
        ntdata = time_data_wBoB[2]
        sptdata = time_data_wBoB[3]

    else:
        sdata = data_eBoB[0]
        pdata = data_eBoB[1]
        ndata = data_eBoB[2]
        spdata = data_eBoB[3]
        
        stdata = time_data_eBoB[0]
        ptdata = time_data_eBoB[1]
        ntdata = time_data_eBoB[2]
        sptdata = time_data_eBoB[3]

    # O2
    x1 = sdata[~np.isnan(sdata)]
    x2 = pdata[~np.isnan(pdata)]
    x3 = ndata[~np.isnan(ndata)]
    x4 = spdata[~np.isnan(spdata)]

    x1t = stdata[~np.isnan(sdata)]
    x2t = ptdata[~np.isnan(pdata)]
    x3t = ntdata[~np.isnan(ndata)]
    x4t = sptdata[~np.isnan(spdata)]

    # get seasonal indicies
    ind_sf = (x1t == 6) | (x1t == 7) | (x1t == 8) | (x1t == 9) | (x1t == 10) | (x1t == 11)
    ind_ws = (x1t == 12) | (x1t == 1) | (x1t == 2) | (x1t == 3) | (x1t == 4) | (x1t ==5)

    x1_sf = x1[ind_sf]
    x1_ws = x1[ind_ws]

    ind_sf = (x2t == 6) | (x2t == 7) | (x2t == 8) | (x2t == 9) | (x2t == 10) | (x2t == 11)
    ind_ws = (x2t == 12) | (x2t == 1) | (x2t == 2) | (x2t == 3) | (x2t == 4) | (x2t ==5)

    x2_sf = x2[ind_sf]
    x2_ws = x2[ind_ws]

    ind_sf = (x3t == 6) | (x3t == 7) | (x3t == 8) | (x3t == 9) | (x3t == 10) | (x3t == 11)
    ind_ws = (x3t == 12) | (x3t == 1) | (x3t == 2) | (x3t == 3) | (x3t == 4) | (x3t ==5)

    x3_sf = x3[ind_sf]
    x3_ws = x3[ind_ws]
    
    ind_sf = (x4t == 6) | (x4t == 7) | (x4t == 8) | (x4t == 9) | (x4t == 10) | (x4t == 11)
    ind_ws = (x4t == 12) | (x4t == 1) | (x4t == 2) | (x4t == 3) | (x4t == 4) | (x4t ==5)

    x4_sf = x4[ind_sf]
    x4_ws = x4[ind_ws]

    # summer/fall --------------------------------------------------------------- #
    ax = fig.add_subplot(2,4,cnt)
    ax.axvspan(-100, hyp_thresh, alpha=0.5, color='silver', linewidth = 0)
    ax.axvline(x = hyp_thresh,color = 'silver',linestyle = '--', linewidth = 0.35)
    ax.axvline(x = 80,color = 'silver',linestyle = '--', linewidth = 0.35)
    
    
    # seasonal 
    tmp = np.sort(x1_sf)
    if len(tmp)>1:
        kernel = stats.gaussian_kde(tmp)
        plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[0])
    
    # positive IOD
    tmp = np.sort(x2_sf)
    if len(tmp)>1:
        kernel = stats.gaussian_kde(tmp)
        plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[1])
    
    # negative IOD
    tmp = np.sort(x3_sf)
    if len(tmp)>1:
        kernel = stats.gaussian_kde(tmp)
        plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[2])

    
    if cnt == 1:
        ax.set_ylabel('PDF \nSummer/Fall', fontsize = 4,labelpad = 1.5)
    else:
        ax.set_yticklabels([])
        
    ax.set_title(regions[pp], pad = 3)
    ax.set_xticklabels([])
        
    ax.set_xlim([0,200])
    ax.set_ylim([0,0.015])
    add_letter(ax, letters[cnt-1], x = 0.01,y=0.91, fontsize = lfz)

    # winter/spring --------------------------------------------------------------- #
    ax = fig.add_subplot(2,4,cnt + 4)
    ax.axvspan(-100, hyp_thresh, alpha=0.5, color='silver', linewidth = 0,label = labels[3])
    ax.axvline(x = hyp_thresh,color = 'silver',linestyle = '--', linewidth = 0.35)
    ax.axvline(x = 80,color = 'silver',linestyle = '--', linewidth = 0.35)

        # seasonal 
    tmp = np.sort(x1_ws)
    if len(tmp)>1:
        kernel = stats.gaussian_kde(tmp)
        plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[0], label = labels[0])
    
    # positive IOD
    tmp = np.sort(x2_ws)
    if len(tmp)>1:
        kernel = stats.gaussian_kde(tmp)
        plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[1], label = labels[1])
    
    # negative IOD
    tmp = np.sort(x3_ws)
    if len(tmp)>1:
        kernel = stats.gaussian_kde(tmp)
        plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[2], label = labels[2])
    
#     # strong positive IOD 
#     tmp = np.sort(x4_ws)
#     if len(tmp)>1:
#         kernel = stats.gaussian_kde(tmp)
#         plt.plot(tmp,kernel(tmp), linewidth = lw, color = clrs[1], linestyle = '--')
        
    ax.set_xlim([0,200])
    ax.set_ylim([0,0.015])
    if cnt == 1:
        ax.set_ylabel('PDF \nWinter/Spring', fontsize = 4,labelpad = 1.5)
    else:
        ax.set_yticklabels([])
    ax.set_xlabel(Depth + ' m [$O_2$] ($\mu mol/ kg$)')
    add_letter(ax, letters[cnt+2], x = 0.01,y=0.91, fontsize = lfz)
    
    cnt += 1

ax.legend(loc='lower center', bbox_to_anchor=(-1.35, -1),
          ncol=5, handlelength = 2.5)

plt.subplots_adjust(wspace = 0.15, bottom = 0.3)

# plt.savefig(figpath + 'Fig_pdfs.pdf', format='pdf', dpi = 400)


# Difference in upwelling for strong posIOD phases and all posIOD phases

In [ ]:
# SLA
ds_SLA = xr.open_dataset('../data_processing/2_SLA/sla_processed.nc')
ds_SLA

mon_sla_mon_anom = ds_SLA['mon_sla_mon_anom']
mon_sla_mon_clim = ds_SLA['mon_sla_mon_clim']
lat = mon_sla_mon_anom.lat.values
lon = mon_sla_mon_anom.lon.values

# load DMI data
ds_DMI= xr.open_dataset('../data_processing/3_DMI/dmi_processed.nc')
posIODyears = list(np.array(ds_DMI.pos_IOD_years)) 
sposIODyears = list(np.array(ds_DMI.spos_IOD_years)) 

# average over the positive IOD years
posIOD_mon_sla_mon_anom,_ = IOD_year_group_grid(mon_sla_mon_anom,IODyear_begin,IODyear_end,posIODyears)
# average over the strong positive IOD years
sposIOD_mon_sla_mon_anom,_ = IOD_year_group_grid(mon_sla_mon_anom,IODyear_begin,IODyear_end,sposIODyears)
# annual cycle
mon_sla_mon_clim = mon_sla_mon_clim.roll(month=-5,roll_coords = False)

# create list of integer years
IODphases = list([mon_sla_mon_clim,
                  sposIOD_mon_sla_mon_anom,
                  posIOD_mon_sla_mon_anom])
titles = ['Strong pIOD Upwelling Increase \nRelative to All pIOD', 
          'Strong pIOD Downwelling Increase \nRelative to All pIOD']

fig = plt.figure(figsize=(14, 6), dpi = 200)

cmin = 0
cmax = .22

params = {'legend.fontsize': 8,
         'axes.labelsize': 14,
         'axes.titlesize': 14,
         'xtick.labelsize':12,
         'ytick.labelsize':12}

pylab.rcParams.update(params)

letters = ['a','b','c','d']

#########################
for ii in range(2):
    
    # Get times and make array of datetime objects
    vtimes = mon_sla_mon_clim.month
    
    data1 = np.zeros([vtimes.shape[0],ds_SLA.sta_loninds.shape[0]])
    data2 = np.zeros([vtimes.shape[0],ds_SLA.sta_loninds.shape[0]])
    data3 = np.zeros([vtimes.shape[0],ds_SLA.sta_loninds.shape[0]])
    for jj in range(ds_SLA.sta_loninds.shape[0]):
        data1[:,jj] = sposIOD_mon_sla_mon_anom[:,ds_SLA.sta_latinds[jj],ds_SLA.sta_loninds[jj]]
        data2[:,jj] = posIOD_mon_sla_mon_anom[:,ds_SLA.sta_latinds[jj],ds_SLA.sta_loninds[jj]]
        data3[:,jj] = mon_sla_mon_clim[:,ds_SLA.sta_latinds[jj],ds_SLA.sta_loninds[jj]]
        
    # data1 = seasonal cycle    |   data2 = all pIODs     |    data3 = spIODs

    if ii == 0: # extra upwelling in spIODs compared to all  pIODs
        data = (np.abs(data1)-np.abs(data2))

        ind = (data1 < 0) & (data2 < 0) & (data > 0)
        data[~ind] = np.nan
    else:# extra downwelling in spIODs compared to all  pIODs
        data = (np.abs(data1)-np.abs(data2))
        
        ind = (data1 > 0) & (data2 > 0) & (data > 0)
        data[~ind] = np.nan
    
    # colorbar limits
    levels = np.round(np.linspace(cmin, cmax, 20),2)

    # Specify longitude values for chosen domain
    sta = np.arange(len(ds_SLA.sta_loninds))

    ax = fig.add_subplot(1,2,ii+1)

    cf = ax.contourf(sta,vtimes,data,levels = levels,cmap=plt.cm.cubehelix_r, extend="both")


    for loc in ds_SLA.loc_list:
        plt.axvline(x=loc,color = 'k',linestyle = ':', linewidth = 2) 
    plt.xlabel('Station', labelpad = 10)
    if (ii == 0):
        plt.ylabel('Month', labelpad = 10)
    
    plt.title(titles[ii], pad = 10)
    if ii == 0:
        ax.set_yticklabels(['Jun','Jul','Aug','Sep','Oct','Nov','Dec','Jan','Feb','Mar', 'Apr', 'May'])
    else:
        ax.set_yticklabels([])
    xticks = (np.array(ds_SLA.loc_list[:-1]) + np.array(ds_SLA.loc_list[1:]))*0.5
    ax.set_xticks(xticks)
    ax.set_xticklabels(['EQ', 'EBoB','WBoB', 'EAS','WAS'])
    ax.tick_params(axis='x', which='major', pad=10)
    ax.set_yticks(list(np.arange(1,13)))
    cf.set_clim(cmin, cmax)# reset lims because contourf does weird things.
#     add_letter(ax, letters[ii], x = 0.01,y=0.94, fontsize = 26)

cbar_ax = fig.add_axes([0.925, 0.125, 0.015, 0.75])
cbar = fig.colorbar(cf,cax=cbar_ax, pad=0.04)
cbar.set_label('$m$', labelpad = 15)

plt.subplots_adjust(wspace = 0.15, right = 0.9)

# plt.savefig(figpath + 'Fig_upwelling_spiod_vs_piod.pdf', format='pdf', dpi = 400)

 # Neutral IOD Phases

In [ ]:
# SLA
ds_SLA = xr.open_dataset('../data_processing/2_SLA/sla_processed.nc')
mon_sla_mon_anom = ds_SLA['mon_sla_mon_anom']
lat = mon_sla_mon_anom.lat.values
lon = mon_sla_mon_anom.lon.values

# load DMI data
ds_DMI= xr.open_dataset('../data_processing/3_DMI/dmi_processed.nc')

neuIODyears = list(np.array(ds_DMI.neu_IOD_years)) 

# average over the neutral IOD years -------------------------------------------# 
neuIOD_mon_sla_mon_anom,_ = IOD_year_group_grid(mon_sla_mon_anom,IODyear_begin,IODyear_end,neuIODyears)

# create list of integer years
IODphases = list([neuIOD_mon_sla_mon_anom])
titles = ['Neutral Phase']
# plt.rcParams.update({'font.size': 20})

fig = plt.figure(figsize=(4, 6), dpi = 200)

cmin = -0.15
cmax = 0.15

letters = ['a','b','c','d','e','f']

params = {'legend.fontsize': 6,
         'axes.labelsize': 8,
         'axes.titlesize': 8.15,
         'xtick.labelsize':6.15,
         'ytick.labelsize':6.15,
         'hatch.linewidth':0.5,
         'hatch.color':'#3A3B3C'}

pylab.rcParams.update(params)

#########################

for ii,phase in enumerate(IODphases):
    
    # Get times and make array of datetime objects
    vtimes = phase.month
    
    data = np.zeros([vtimes.shape[0],ds_SLA.sta_loninds.shape[0]])
    ac = np.zeros([vtimes.shape[0],ds_SLA.sta_loninds.shape[0]])
    
    for jj in range(ds_SLA.sta_loninds.shape[0]):
        data[:,jj] = phase[:,ds_SLA.sta_latinds[jj],ds_SLA.sta_loninds[jj]]

    # colorbar limits
    levels = np.round(np.linspace(cmin, cmax, 10),2)

    # Specify longitude values for chosen domain
    sta = np.arange(len(ds_SLA.sta_loninds))

    ax = fig.add_subplot(1,1,ii+1)

    cf = ax.contourf(sta,vtimes,data,levels = levels,cmap=plt.cm.PuOr_r, extend="both")

    
    for loc in ds_SLA.loc_list:
        plt.axvline(x=loc,color = 'k', linewidth = 0.5)
        
#     plt.xlabel('Station')
    if ii == 0:
        plt.ylabel('Month')
    
    plt.title(titles[ii])
    if ii == 0:
        ax.set_yticklabels(['Jun','Jul','Aug','Sep','Oct','Nov','Dec','Jan','Feb','Mar', 'Apr', 'May'])
    else:
        ax.set_yticklabels([])
        
    xticks = (np.array(ds_SLA.loc_list[:-1]) + np.array(ds_SLA.loc_list[1:]))*0.5
    ax.set_xticks(xticks)
    ax.set_xticklabels(['EQ', 'EBoB','WBoB', 'EAS','WAS'])
    ax.tick_params(axis='x', which='major', pad=10)
    ax.set_yticks(list(np.arange(1,13)))
    cf.set_clim(cmin, cmax)# reset lims because contourf does weird things.
#     add_letter(ax, letters[ii], x = 0.01,y=0.955, fontsize = 7)

# cbar_ax = fig.add_axes([0.91, 0.125, 0.015, 0.75])
cbar_ax = fig.add_axes([0.125, 0.2, 0.775, 0.025])
cbar = fig.colorbar(cf,cax=cbar_ax, pad=0.04, orientation = 'horizontal')
cbar.set_label('Interannual Sea-level Anomaly ($m$)')

plt.subplots_adjust(wspace = 0.15, bottom = 0.3)

# plt.savefig(figpath + 'Fig_sla_neutral.pdf', format='pdf', dpi = 400)